# Impostazione

## python version

In [1]:
from platform import python_version
print(python_version())

3.7.9


## Librerie

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup, Comment
import mechanicalsoup as ms
import scrapy
import selenium
import lxml
import re
import requests
from requests import get

# Pokemon list with stats (id, nome, tipo, total, hp, attack, defense, sp. atk., sp. def, speed)

Url della risorsa, HTTPResponse Object e BeautifulSoupObject della pagina

In [2]:
URL = "https://pokemondb.net/pokedex/all"
pokemon_list_page = requests.get(URL)

In [4]:
pokemon_list_soup = BeautifulSoup(pokemon_list_page.content, features="html.parser")

Estraggo la sola tabella

In [5]:
pokemon_list_table = pokemon_list_soup.find("div", class_ = "grid-col")

In [6]:
pokemon_list_table = pokemon_list_soup.find("tbody")

In [9]:
print(pokemon_list_table.prettify())

<tbody>
 <tr>
  <td class="cell-num cell-fixed" data-sort-value="1">
   <span class="infocard-cell-img">
    <span class="img-fixed icon-pkmn" data-alt="Bulbasaur icon" data-src="https://img.pokemondb.net/sprites/sword-shield/icon/bulbasaur.png">
    </span>
   </span>
   <span class="infocard-cell-data">
    001
   </span>
  </td>
  <td class="cell-name">
   <a class="ent-name" href="/pokedex/bulbasaur" title="View Pokedex for #001 Bulbasaur">
    Bulbasaur
   </a>
  </td>
  <td class="cell-icon">
   <a class="type-icon type-grass" href="/type/grass">
    Grass
   </a>
   <br/>
   <a class="type-icon type-poison" href="/type/poison">
    Poison
   </a>
  </td>
  <td class="cell-total">
   318
  </td>
  <td class="cell-num">
   45
  </td>
  <td class="cell-num">
   49
  </td>
  <td class="cell-num">
   49
  </td>
  <td class="cell-num">
   65
  </td>
  <td class="cell-num">
   65
  </td>
  <td class="cell-num">
   45
  </td>
 </tr>
 <tr>
  <td class="cell-num cell-fixed" data-sort-valu

Vado a estrarre i singoli elmenti relativi ai diversi pokemon

In [18]:
pokemon_lists_elements = pokemon_list_table.find_all("tr")

Qualcosa non va, il numero di elementi nella tabella non combacia con il numero di pokemon del pokedex:
- A quanto pare il problema è che le megaevoluzioni e le varianti hanno lo stesso id

In [23]:

print(len(pokemon_lists_elements))

1075


In [33]:
counter = 1
for pokemon_lists_element in pokemon_lists_elements:
    pokedex_number = pokemon_lists_element.find("span", class_ = "infocard-cell-data").text
    print(int(pokedex_number))
    print(counter)
    print(int(pokedex_number) == counter)
    counter += 1

1
1
True
2
2
True
3
3
True
3
4
False
4
5
False
5
6
False
6
7
False
6
8
False
6
9
False
7
10
False
8
11
False
9
12
False
9
13
False
10
14
False
11
15
False
12
16
False
13
17
False
14
18
False
15
19
False
15
20
False
16
21
False
17
22
False
18
23
False
18
24
False
19
25
False
19
26
False
20
27
False
20
28
False
21
29
False
22
30
False
23
31
False
24
32
False
25
33
False
25
34
False
26
35
False
26
36
False
27
37
False
27
38
False
28
39
False
28
40
False
29
41
False
30
42
False
31
43
False
32
44
False
33
45
False
34
46
False
35
47
False
36
48
False
37
49
False
37
50
False
38
51
False
38
52
False
39
53
False
40
54
False
41
55
False
42
56
False
43
57
False
44
58
False
45
59
False
46
60
False
47
61
False
48
62
False
49
63
False
50
64
False
50
65
False
51
66
False
51
67
False
52
68
False
52
69
False
52
70
False
53
71
False
53
72
False
54
73
False
55
74
False
56
75
False
57
76
False
58
77
False
58
78
False
59
79
False
59
80
False
60
81
False
61
82
False
62
83
False
63
84
False
64
85
False
65
86